In [3]:
import csv
import mapreduce as mr
import dateutil
from dateutil import parser

In [10]:
#Loading data

data = pd.read_csv('311_noise.csv')
data.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Status,Due Date,Resolution Description,X Coordinate (State Plane),Y Coordinate (State Plane),Latitude,Longitude,Location
0,33047245,04/03/2016 02:11:01 AM,NaN,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10032,558 WEST 164 STREET,Open,04/03/2016 10:11:01 AM,Your complaint has been forwarded to the New Y...,1000650,244594,40.838014,-73.940731,"(40.83801407976737, -73.94073148488742)"
1,33047210,04/03/2016 02:08:59 AM,NaN,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Club/Bar/Restaurant,10009,NaN,Open,04/03/2016 10:08:59 AM,Your complaint has been forwarded to the New Y...,987457,202780,40.723260,-73.988430,"(40.723260387178414, -73.98843007160804)"
2,33043665,04/03/2016 02:07:15 AM,NaN,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11229,1864 EAST 14 STREET,Open,04/03/2016 10:07:15 AM,Your complaint has been forwarded to the New Y...,995802,159441,40.604298,-73.958398,"(40.604297503438865, -73.9583979847671)"
3,33042520,04/03/2016 02:05:23 AM,NaN,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Store/Commercial,11206,145 MANHATTAN AVENUE,Assigned,04/03/2016 10:05:23 AM,Your complaint has been received by the Police...,999498,197152,40.707800,-73.945002,"(40.707800316637616, -73.94500238123231)"
4,33042106,04/03/2016 02:05:16 AM,04/03/2016 02:44:44 AM,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Store/Commercial,11234,2192 FLATBUSH AVENUE,Closed,04/03/2016 10:05:16 AM,The Police Department responded to the complai...,1003628,163910,40.616550,-73.930202,"(40.61655032892211, -73.93020153359745)"


#1. Weekday

In [4]:
#Calculating sum of complaints per zipcode per weekday

def mapper1(row):
    weekday = (row['Incident Zip'], dateutil.parser.parse(row['Created Date']).weekday())
    yield (weekday, 1)

def reducer1(k2v2):
    weekday, count = k2v2
    return (weekday, sum(count)) 

with open('noise_test.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output1 = list(mr.run(reader, mapper1, reducer1))

output1[6:11]

[(('10001', 0), 6),
 (('10001', 1), 12),
 (('10001', 2), 8),
 (('10001', 4), 15),
 (('10001', 5), 15)]

In [10]:
#Getting unique zipcodes

zipcodes = []

for i in output1:
    zipcodes.append(i[0][0])
    
zipcodes = list(set(zipcodes))

zipcodes[:10]

['',
 '10065',
 '10453',
 '10452',
 '10451',
 '10457',
 '10456',
 '10455',
 '10454',
 '10459']

In [18]:
#Creating dictionaries for each zipcode

weekday = {}

for i in zipcodes:
    weekday[i] = []
    
for i in output1:
    weekday[i[0][0]].append(((i[0][1]), i[1]))
    
weekday['11216']

[(0, 7), (1, 9), (2, 10), (4, 26), (5, 30), (6, 20)]

#2. Hour

In [27]:
#Calculating sum of complaints per zipcode per hour

def mapper1(row):
    hour = (row['Incident Zip'], dateutil.parser.parse(row['Created Date']).hour)
    yield (hour, 1)

def reducer1(k2v2):
    hour, count = k2v2
    return (hour, sum(count)) 

with open('noise_test.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output2 = list(mr.run(reader, mapper1, reducer1))

output2[15:20]

[(('10001', 0), 8),
 (('10001', 1), 7),
 (('10001', 2), 5),
 (('10001', 4), 2),
 (('10001', 6), 1)]

In [28]:
#Creating dictionaries for each zipcode

hour = {}

for i in zipcodes:
    hour[i] = []
    
for i in output2:
    hour[i[0][0]].append(((i[0][1]), i[1]))
    
hour['11216']

[(0, 10),
 (1, 23),
 (2, 4),
 (3, 8),
 (4, 3),
 (5, 1),
 (7, 4),
 (9, 3),
 (10, 1),
 (11, 4),
 (12, 1),
 (14, 1),
 (16, 3),
 (17, 3),
 (18, 2),
 (19, 6),
 (20, 3),
 (21, 6),
 (22, 3),
 (23, 13)]

#3. Descriptor